# Case Técnico - H:Data // Matheus Tenório

## 1. Importação de Bibliotecas e Configurações

In [156]:
import pandas as pd

In [157]:
pd.options.display.max_columns = 200

## 2. Coleta de informações

In [158]:
origin = pd.read_excel('../data/finance-data-received.xlsx') # Aba principal da planilha
normalization_accounts = pd.read_excel('../data/finance-data-received.xlsx', sheet_name='DEPARA_CONTAS_FINANC') # Aba de normalização de contas
normalization_costs = pd.read_excel('../data/finance-data-received.xlsx', sheet_name='DEPARA_CENTRO_CUSTOS') # Aba de normalização de centro de custos

In [159]:
origin.shape

(3535, 15)

In [160]:
origin.head(10)

,ID,CONTA,CENTRO DE CUSTO,MÊS REFERÊNCIA,DATA VENCIMENTO,FORNECEDOR,NOTA FISCAL,VALOR TOTAL,VALOR PARCELA,CONSIDERAÇÕES LANÇAMENTO,STATUS,DATA PAGAMENTO,VALOR REALIZADO,CONTA BANCARIA,FORMA DE PAGAMENTO
0,6.0,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,Farmacia,2023-03-01,2023-04-28,FORNECEDOR 444,18185,15.00,15.00,Materiais Descartaveis Cozinha,PAGO,2023-04-01,15.00,GERAL,NaN
1,7.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Tecnologia Informacao,2023-03-01,2023-03-10,FORNECEDOR 183,136874,36.53,36.53,Material de manutencao predial,PAGO,2023-03-01,36.53,GERAL,NaN
2,8.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-03-27,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-03-01,328.58,GERAL,NaN
3,9.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,UTI,2023-03-01,2023-04-24,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-04-01,328.58,GERAL,NaN
4,10.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-05-22,FORNECEDOR 183,137186,328.58,328.58,Fechadura,PAGO,2023-05-01,328.58,GERAL,NaN
5,11.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-06-19,FORNECEDOR 183,137186,328.56,328.56,Material de manutencao predial,PAGO,2023-06-01,328.56,GERAL,NaN
6,12.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Pronto Atendimento,2023-03-01,2023-05-22,FORNECEDOR 183,138556,120.90,120.90,Fechadura,PAGO,2023-05-01,120.90,GERAL,NaN
7,13.0,MATERIAL HIGIENIZACAO (CONSUMO DE MATERIAIS GE...,Ambulatorio,2023-03-01,2023-04-21,FORNECEDOR 416,2680,280.04,280.04,Material de limpeza e higiene,PAGO,2023-04-01,280.04,GERAL,NaN
8,14.0,SERV.OBRAS E REFORMAS (MANUTENCAO E ASSISTENCI...,Geral,2023-03-01,2023-01-04,FORNECEDOR 27,20231,3120.00,3120.00,REFORMA,PAGO,2023-01-01,3120.00,GERAL,NaN
9,15.0,SERV.OBRAS E REFORMAS (MANUTENCAO E ASSISTENCI...,Pronto Atendimento,2023-03-01,2023-01-09,FORNECEDOR 27,2023 02,4310.00,4310.00,REFORMA,PAGO,2023-01-01,4310.00,GERAL,NaN


In [161]:
origin.columns

Index(['ID', 'CONTA', 'CENTRO DE CUSTO', 'MÊS REFERÊNCIA', 'DATA VENCIMENTO',
       'FORNECEDOR', 'NOTA FISCAL', 'VALOR TOTAL', 'VALOR PARCELA',
       'CONSIDERAÇÕES LANÇAMENTO', 'STATUS', 'DATA PAGAMENTO',
       'VALOR REALIZADO', 'CONTA BANCARIA', 'FORMA DE PAGAMENTO'],
      dtype='object')

In [162]:
origin.dtypes

ID                                 float64
CONTA                               object
CENTRO DE CUSTO                     object
MÊS REFERÊNCIA              datetime64[ns]
DATA VENCIMENTO             datetime64[ns]
FORNECEDOR                          object
NOTA FISCAL                         object
VALOR TOTAL                        float64
VALOR PARCELA                      float64
CONSIDERAÇÕES LANÇAMENTO            object
STATUS                              object
DATA PAGAMENTO              datetime64[ns]
VALOR REALIZADO                    float64
CONTA BANCARIA                      object
FORMA DE PAGAMENTO                 float64
dtype: object

## 3. Enriquecimento // Tratamento de dados

#### 3.1 Enriquecimento de dados de contas

In [163]:
# Criação de duas novas colunas, Categoria da Conta e ID Categoria da Conta, visando trazer os dados da aba de normalização de contas
accounts_index = list(origin.columns).index('CONTA')

origin.insert(accounts_index + 1, 'ID Categoria Conta', '')
origin.insert(accounts_index + 2, 'Categoria Conta', '')

In [164]:
# Loop for para popular as duas colunas baseado nos dados que temos na aba de normalização de contas

for index, row in origin.iterrows():
    if row['CONTA'] in normalization_accounts['ds_conta_financeira'].values:
        match = normalization_accounts[normalization_accounts['ds_conta_financeira'] == row['CONTA']]
        
        category = match.iloc[0]['ds_normal_conta_financeira']
        category_id = match.iloc[0]['sk_normal_conta_financeira']
        
        origin.at[index, 'Categoria Conta'] = category
        origin.at[index, 'ID Categoria Conta'] = category_id

In [165]:
# Checagem do resultado
origin.head(10)

,ID,CONTA,ID Categoria Conta,Categoria Conta,CENTRO DE CUSTO,MÊS REFERÊNCIA,DATA VENCIMENTO,FORNECEDOR,NOTA FISCAL,VALOR TOTAL,VALOR PARCELA,CONSIDERAÇÕES LANÇAMENTO,STATUS,DATA PAGAMENTO,VALOR REALIZADO,CONTA BANCARIA,FORMA DE PAGAMENTO
0,6.0,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,11,CONSUMO DE MATERIAIS GERAIS,Farmacia,2023-03-01,2023-04-28,FORNECEDOR 444,18185,15.00,15.00,Materiais Descartaveis Cozinha,PAGO,2023-04-01,15.00,GERAL,NaN
1,7.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Tecnologia Informacao,2023-03-01,2023-03-10,FORNECEDOR 183,136874,36.53,36.53,Material de manutencao predial,PAGO,2023-03-01,36.53,GERAL,NaN
2,8.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Ambulatorio,2023-03-01,2023-03-27,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-03-01,328.58,GERAL,NaN
3,9.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,UTI,2023-03-01,2023-04-24,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-04-01,328.58,GERAL,NaN
4,10.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Ambulatorio,2023-03-01,2023-05-22,FORNECEDOR 183,137186,328.58,328.58,Fechadura,PAGO,2023-05-01,328.58,GERAL,NaN
5,11.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Ambulatorio,2023-03-01,2023-06-19,FORNECEDOR 183,137186,328.56,328.56,Material de manutencao predial,PAGO,2023-06-01,328.56,GERAL,NaN
6,12.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Pronto Atendimento,2023-03-01,2023-05-22,FORNECEDOR 183,138556,120.90,120.90,Fechadura,PAGO,2023-05-01,120.90,GERAL,NaN
7,13.0,MATERIAL HIGIENIZACAO (CONSUMO DE MATERIAIS GE...,11,CONSUMO DE MATERIAIS GERAIS,Ambulatorio,2023-03-01,2023-04-21,FORNECEDOR 416,2680,280.04,280.04,Material de limpeza e higiene,PAGO,2023-04-01,280.04,GERAL,NaN
8,14.0,SERV.OBRAS E REFORMAS (MANUTENCAO E ASSISTENCI...,14,MANUTENCAO E ASSISTENCIA TECNICA,Geral,2023-03-01,2023-01-04,FORNECEDOR 27,20231,3120.00,3120.00,REFORMA,PAGO,2023-01-01,3120.00,GERAL,NaN
9,15.0,SERV.OBRAS E REFORMAS (MANUTENCAO E ASSISTENCI...,14,MANUTENCAO E ASSISTENCIA TECNICA,Pronto Atendimento,2023-03-01,2023-01-09,FORNECEDOR 27,2023 02,4310.00,4310.00,REFORMA,PAGO,2023-01-01,4310.00,GERAL,NaN


#### 3.2 Enriquecimento de dados de centro de custos

In [166]:
# Criação de duas novas colunas, Categoria CC e ID Categoria CC, visando trazer os dados da aba de normalização de contas
costs_index = list(origin.columns).index('CENTRO DE CUSTO')

origin.insert(costs_index + 1, 'ID Categoria CC', '')

In [167]:
# Loop for para popular as duas colunas baseado nos dados que temos na aba de normalização de contas

for index, row in origin.iterrows():
    if row['CENTRO DE CUSTO'] in normalization_costs['ds_normal_centro_custo'].values:
        match = normalization_costs[normalization_costs['ds_normal_centro_custo'] == row['CENTRO DE CUSTO']]
        
        category_id = match.iloc[0]['sk_normal_centro_custo']
        
        origin.at[index, 'ID Categoria CC'] = category_id

In [168]:
origin.head(10)

,ID,CONTA,ID Categoria Conta,Categoria Conta,CENTRO DE CUSTO,ID Categoria CC,MÊS REFERÊNCIA,DATA VENCIMENTO,FORNECEDOR,NOTA FISCAL,VALOR TOTAL,VALOR PARCELA,CONSIDERAÇÕES LANÇAMENTO,STATUS,DATA PAGAMENTO,VALOR REALIZADO,CONTA BANCARIA,FORMA DE PAGAMENTO
0,6.0,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,11,CONSUMO DE MATERIAIS GERAIS,Farmacia,8,2023-03-01,2023-04-28,FORNECEDOR 444,18185,15.00,15.00,Materiais Descartaveis Cozinha,PAGO,2023-04-01,15.00,GERAL,NaN
1,7.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Tecnologia Informacao,7,2023-03-01,2023-03-10,FORNECEDOR 183,136874,36.53,36.53,Material de manutencao predial,PAGO,2023-03-01,36.53,GERAL,NaN
2,8.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Ambulatorio,3,2023-03-01,2023-03-27,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-03-01,328.58,GERAL,NaN
3,9.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,UTI,5,2023-03-01,2023-04-24,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-04-01,328.58,GERAL,NaN
4,10.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Ambulatorio,3,2023-03-01,2023-05-22,FORNECEDOR 183,137186,328.58,328.58,Fechadura,PAGO,2023-05-01,328.58,GERAL,NaN
5,11.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Ambulatorio,3,2023-03-01,2023-06-19,FORNECEDOR 183,137186,328.56,328.56,Material de manutencao predial,PAGO,2023-06-01,328.56,GERAL,NaN
6,12.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,14,MANUTENCAO E ASSISTENCIA TECNICA,Pronto Atendimento,1,2023-03-01,2023-05-22,FORNECEDOR 183,138556,120.90,120.90,Fechadura,PAGO,2023-05-01,120.90,GERAL,NaN
7,13.0,MATERIAL HIGIENIZACAO (CONSUMO DE MATERIAIS GE...,11,CONSUMO DE MATERIAIS GERAIS,Ambulatorio,3,2023-03-01,2023-04-21,FORNECEDOR 416,2680,280.04,280.04,Material de limpeza e higiene,PAGO,2023-04-01,280.04,GERAL,NaN
8,14.0,SERV.OBRAS E REFORMAS (MANUTENCAO E ASSISTENCI...,14,MANUTENCAO E ASSISTENCIA TECNICA,Geral,6,2023-03-01,2023-01-04,FORNECEDOR 27,20231,3120.00,3120.00,REFORMA,PAGO,2023-01-01,3120.00,GERAL,NaN
9,15.0,SERV.OBRAS E REFORMAS (MANUTENCAO E ASSISTENCI...,14,MANUTENCAO E ASSISTENCIA TECNICA,Pronto Atendimento,1,2023-03-01,2023-01-09,FORNECEDOR 27,2023 02,4310.00,4310.00,REFORMA,PAGO,2023-01-01,4310.00,GERAL,NaN


## Exportação de dados para tabela única

In [169]:
origin.to_excel('../data/Tabelão.xlsx', index=False)

## Exportação de dados DW // PostgreSQL